# Models built for data prediction

In [33]:
from _Setup import *

In [34]:
# Import Data
train = pd.read_csv(missForestTrainFile)
val = pd.read_csv(validationFile)
test = pd.read_csv(missForestTestFile)

In [35]:
X_train = train.drop(["Unit2", "HospAdmTime", "Unnamed: 0"], axis = 1)
X_train['SepsisLabel'] = X_train['SepsisLabel'].astype(int)
y_train = train['SepsisLabel']

val = val.drop(["Unit2", "HospAdmTime", "Unnamed: 0"], axis = 1)
val['SepsisLabel'] = val['SepsisLabel'].astype(int)

test = test.drop(["Unit2", "HospAdmTime"], axis = 1)

In [36]:
train.head()

,Unnamed: 0,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,...,Shock,patient,Gender,Unit1,Unit2,Fever,TCP,TCA,LEU,SepsisLabel
0,0,-0.799514,-0.449283,0.743334,-2.128854,-0.614444,0.156049,0.427405,-1.296970,0.041034,...,-0.042956,16426,1.0,0,1,1,0,0,1,0.0
1,1,-0.775449,-0.449283,0.743334,-2.128854,-0.614444,0.156049,0.427405,-1.296970,0.041034,...,-0.042956,16426,1.0,0,1,1,0,0,1,0.0
2,2,-0.751383,-0.338220,0.587902,-0.930922,-0.421269,0.310043,0.723030,-1.296970,0.041034,...,-0.093116,16426,1.0,0,1,1,0,0,1,0.0
3,3,-0.727317,-0.393751,0.898766,-0.552628,-0.464197,0.063652,0.353499,-1.296970,-0.752669,...,-0.105082,16426,1.0,0,1,1,0,0,1,0.0
4,4,-0.703251,-0.726942,0.898766,-0.552628,-0.335414,-0.121140,0.057875,-1.480397,-0.752669,...,-0.428978,16426,1.0,0,1,1,0,0,1,0.0


In [37]:
test.head()

,Unnamed: 0,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,...,Age,ICULOS,Shock,patient,Gender,Unit1,Fever,TCP,TCA,LEU
0,0,-0.799514,1.327732,0.277039,0.922720,0.050936,-0.244336,-0.359181,-0.56326,0.025854,...,1.201721,-0.786585,0.728995,6560,1.0,1.0,0,0,1,0
1,1,-0.775449,0.939010,-0.344688,0.922720,-0.936402,-0.757443,-0.610754,-0.56326,0.019334,...,1.201721,-0.762683,1.338033,6560,1.0,1.0,0,0,1,0
2,2,-0.751383,1.050073,-0.344688,0.922720,0.093863,-0.387858,-0.487018,-0.56326,0.019334,...,1.201721,-0.738780,0.512848,6560,1.0,1.0,0,0,1,0
3,3,-0.727317,1.771985,-0.966415,0.922720,1.167056,0.782496,0.499878,-0.56326,0.019334,...,1.201721,-0.714877,0.268080,6560,1.0,1.0,0,0,1,0
4,4,-0.703251,0.716883,-0.966415,0.708353,-1.065185,-1.106702,-0.912023,-0.56326,0.019334,...,1.201721,-0.690975,1.293984,6560,1.0,1.0,0,0,1,0


In [38]:
ydf_model = ydf.GradientBoostedTreesLearner(label='SepsisLabel').train(X_train, verbose = 2)

Train model on 1195994 examples
Model trained in 0:03:57.542041


In [39]:
ydf_model.describe()

In [40]:
model_evaluated = ydf_model.evaluate(val)

In [41]:
model_evaluated

Label \ Pred,0,1
0,212753,3648
1,1540,262


In [42]:
mat = model_evaluated.confusion_matrix
vals = mat.matrix

precision = vals[1][1] / (vals[1][1] + vals[1][0])
recall = vals[1][1] / (vals[1][1] + vals[0][1])

f1 = (2*precision*recall)/(precision+recall)
print(precision, recall, f1)

0.0670076726342711 0.1453940066592675 0.09173669467787114


In [43]:
# Predict labels
yHatTest = ydf_model.predict(test)

In [44]:
yHatTestScaled = np.where(yHatTest >= 0.5, 1, 0)

In [45]:
test_submission = pd.read_csv(rawSubmissionFile)
test_submission.head()

,ID,SepsisLabel
0,6560_1,0
1,6560_2,0
2,6560_3,0
3,6560_4,0
4,6560_5,0


In [46]:
test_submission['IDAppended'] = test_submission['ID'].str.split('_').str[0]
test_submission['SepsisLabel'] = test_submission.groupby('IDAppended')['SepsisLabel'].transform(
    lambda x: (x.cumsum() > 0).astype(int)
)

In [47]:
test_submission = test_submission.drop('IDAppended', axis = 1)
test_submission['SepsisLabel'] = yHatTestScaled
test_submission.head()

,ID,SepsisLabel
0,6560_1,0
1,6560_2,0
2,6560_3,0
3,6560_4,0
4,6560_5,0


In [48]:
test_submission.to_csv("../Data/rawData/ydf_submission.csv", index = False)

In [49]:
test_submission.shape

(464354, 2)